In [1]:
import pandas as pd 
import numpy as np 
import sklearn as sk


# Part 1 Original Stopwords

In [2]:
#Reading In Stop Words 
stopwords=pd.read_csv('/Users/ellisobrien/Desktop/Georgetown Semester 1/Data Science/coding_discussions_ppol564_fall2021/04_coding_discussion/Data/stop_words.csv')

#Creating directory 
folder='/Users/ellisobrien/Desktop/Georgetown Semester 1/Data Science/coding_discussions_ppol564_fall2021/04_coding_discussion/Data'

#reading in each file 
aljazeera=open((str(folder)+"/aljazeera-khashoggi.txt"), "r")
bbc=open((str(folder)+"/bbc-khashoggi.txt"), "r")
breitbart=open((str(folder)+"/breitbart-khashoggi.txt"), "r")
cnn=open((str(folder)+"/cnn-khashoggi.txt"), "r")
fox=open((str(folder)+"/fox-khashoggi.txt"), "r")


In [3]:
#Converting each text to string
aljazeera=aljazeera.read()
bbc=bbc.read()
breitbart=breitbart.read()
cnn=cnn.read()
fox=fox.read()


In [4]:
#importing sklearn count vectorizer 
from sklearn.feature_extraction.text import CountVectorizer
#creating list of texts
textlist = [aljazeera, bbc, breitbart, cnn, fox]
#setting vectorizer
vectorizer = CountVectorizer(stop_words=list(stopwords['word']))

#applying vectorizer 
vf = vectorizer.fit_transform(textlist)

#printing as array
print(vf.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 1 0 ... 2 0 1]]


/Users/ellisobrien/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'mustn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [50]:
#Creating function 
def cosdot(x,y):
    '''
    inputs: two vectors 
    outputs: cosine dotproduct 
    '''
    vfx=vf[x].todense()
    vfy=vf[y].todense()
    dp = np.dot(vfx,vfy.transpose())/(np.sqrt(np.dot(vfx,vfx.transpose())) * np.sqrt(np.dot(vfy,vfy.transpose())))
    return float(dp)

In [54]:
#list of numbers for loop
numberlist=[0,1,2,3,4]
#list of column names for loop
columnnames=['aljazeera', 'bbc', 'breitbart', 'cnn', 'fox']
#blank Data Frame 
df = pd.DataFrame(columns=["Article1", "Article2", "Coefficient"])

In [55]:
#looping through to add each to a data frame 
for i in (numberlist): 
    for j in range(i+1, 5):
         df.loc[len(df)]=[columnnames[i],columnnames[j], cosdot(i, j)]

,Article1,Article2,Coefficient
0,aljazeera,bbc,0.704288
1,aljazeera,breitbart,0.600776
2,aljazeera,cnn,0.608624
3,aljazeera,fox,0.719294
4,bbc,breitbart,0.621047
5,bbc,cnn,0.550651
6,bbc,fox,0.692978
7,breitbart,cnn,0.421972
8,breitbart,fox,0.581209
9,cnn,fox,0.605116


# Part 2 with New Stop Words

In [61]:
#Finding the most common words
words = vf.sum(axis=0)
word_frequency = [(word, words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
word_frequency =sorted(word_frequency, key = lambda x: x[1], reverse=True)
print(word_frequency[:7])

[('saudi', 56), ('erdogan', 49), ('khashoggi', 47), ('consulate', 24), ('turkish', 23), ('arabia', 23), ('murder', 22)]


In [63]:
#Appending New Words
new_stop_words = stopwords['word'].tolist()+['saudi', 'erdogan', 'khashoggi',  'consulate', 'turkish', 'arabia', 'murder']
#Making List to dataframe 
new_stop_words = pd.DataFrame(new_stop_words,columns=['word'])


In [72]:
#setting vectorizer with new stop words
vectorizer2 = CountVectorizer(stop_words=list(new_stop_words['word']))
#creating new object with new stopwords
vf2 = vectorizer2.fit_transform(textlist)
#printing as array
print(vf2.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 1 0 ... 2 0 1]]


/Users/ellisobrien/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'mustn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [88]:
#function with new stopword list
def cosdot2(x,y):
    '''
    inputs: two vectors 
    outputs: cosine dotproduct 
    '''
    vf2x=vf2[x].todense()
    vf2y=vf2[y].todense()
    dp2 = np.dot(vf2x,vf2y.transpose())/(np.sqrt(np.dot(vf2x,vf2x.transpose())) * np.sqrt(np.dot(vf2y,vf2y.transpose())))
    return float(dp2)

In [90]:
#data frame for new stopword lists
df2 = pd.DataFrame(columns=["Article1", "Article2", "New_Coefficient"])

In [91]:
#looping through new stop word array 
for i in (numberlist): 
    for j in range(i+1, 5):
         df2.loc[len(df2)]=[columnnames[i],columnnames[j], cosdot2(i, j)]

In [92]:
#joining two data frames
final_df = pd.merge(df, df2, how="left", on=["Article1", "Article2"])
#printing final data frame 
final_df

,Article1,Article2,Coefficient,New_Coefficient
0,aljazeera,bbc,0.704288,0.457895
1,aljazeera,breitbart,0.600776,0.436066
2,aljazeera,cnn,0.608624,0.342936
3,aljazeera,fox,0.719294,0.422397
4,bbc,breitbart,0.621047,0.480912
5,bbc,cnn,0.550651,0.266810
6,bbc,fox,0.692978,0.431374
7,breitbart,cnn,0.421972,0.211457
8,breitbart,fox,0.581209,0.423251
9,cnn,fox,0.605116,0.266060


### Observations
With the original list the two most similar articles are Aljazeera and Fox. The least similar are Breitbart and CNN. There is much less similarity between the articles once the list of stop words are ammended which makes sense. With the new list the most similar are Breibart and BBC, which marks a change from the original list. The least similar are still Breitbart and CNN. 

Over the correlation with the original stop words is in between .42 and .71. With the new stop words it is betweet .21 and .48. I am not surprised to see CNN and Breitbart as the most dissimiliar as the 
